In [1]:
# Dependencies
import hvplot.pandas

import requests
import pandas as pd
from census import Census
import numpy as np
import matplotlib.pyplot as plt
from config import api_key
import seaborn as sns
import plotly.express as px
import geopandas as gpd
## Defining variables for Census API





## Defining API Call Variables


In [2]:
population = 'B01003_001E'
meidan_income = 'B19013_001E'
years = [2018, 2019, 2021, 2022] #2020 Excluded due to no data because of COVID 
base_url = 'https://api.census.gov/data/'
county_key = '&for=county:*&in=state:06' 
state_key =  '&for=state:06' 


## Prep Work - Pulling in CSV files to identify county/state codes against the key

In [3]:
#Pulling Geocodes for State(CA)
geocodes_csv = "../Josh/Resources/geocodes.csv"
geo_state = pd.read_csv(geocodes_csv)
geo_state = geo_state.rename(columns={"Area Name (including legal/statistical area description)":"Area","State Code (FIPS)":"State"})
geo_state = geo_state[(geo_state['State']== 6) & (geo_state["County Code (FIPS)"]==0) *(geo_state["Place Code (FIPS)"] == 0)]
geo_state = geo_state[["State","Area"]]
geo_state

,State,Area
1393,6,California


In [4]:
#Pulling Geocodes for County(CA)
geocodes_csv = "../Josh/Resources/geocodes.csv"
geo_county = pd.read_csv(geocodes_csv)
geo_county = geo_county.rename(columns={"Area Name (including legal/statistical area description)":"Area","State Code (FIPS)":"State","County Code (FIPS)":"County"})
geo_county = geo_county[['State','County','Area']]
geo_county = geo_county[(geo_county['State']== 6) & (geo_county["County"] > 0)]
geo_county.head()

,State,County,Area
1394,6,1,Alameda County
1395,6,3,Alpine County
1396,6,5,Amador County
1397,6,7,Butte County
1398,6,9,Calaveras County


In [5]:
#Created CSV for Regions in case we want to do Socal/NorCal Analysis
state_split_csv = "../Josh/Resources/California - Counties.csv"
state_split = pd.read_csv(state_split_csv)
state_split.head()

,County,Region
0,Alameda County,NorCal
1,Alpine County,NorCal
2,Amador County,NorCal
3,Butte County,NorCal
4,Calaveras County,NorCal


## Pulling in  Population Data from Census API (ACS)


##### County


In [6]:
dfs = []

for year in years:
    url = f'{base_url}{year}/acs/acs1?get={population}{county_key}&key={api_key}'

    data = pd.DataFrame(requests.get(url).json()[1:])

    data["Year"] = year

    data = data.rename(columns={0: 'Population', 1: 'State', 2: 'County'})

    dfs.append(data)

pop_county = pd.concat(dfs, ignore_index=True)

pop_county['State'] = pd.to_numeric(pop_county['State'])
pop_county['County'] = pd.to_numeric(pop_county['County'])
pop_county["Population"] = pop_county["Population"].astype('int')

pop_county = pd.merge(pop_county,geo_county,on=['State','County'],how='left')
pop_county.drop(columns=["County"], inplace=True)
pop_county.rename(columns={'Area': 'County'},inplace=True)
pop_county['State'] = 'California'
pop_county  = pd.merge(pop_county,state_split, on= "County",how='left')
pop_county = pop_county[['Year', 'State','Region','County','Population']]

pop_county.head()


,Year,State,Region,County,Population
0,2018,California,NorCal,Lake County,64382
1,2018,California,NorCal,Merced County,274765
2,2018,California,NorCal,Contra Costa County,1150215
3,2018,California,NorCal,Stanislaus County,549815
4,2018,California,SoCal,Santa Barbara County,446527


##### State


In [7]:

dfs = []

for year in years: 
    url = f'{base_url}{year}/acs/acs1?get={population}{state_key}&key={api_key}'

    data = requests.get(url).json()

    data = pd.DataFrame(data[1:])
   
    data["Year"] = year

    data = data.rename(columns={0: 'Population',1: 'State',2:'County'})
    dfs.append(data)

pop_state = pd.concat(dfs, ignore_index=True)
pop_state['State'] = pd.to_numeric(pop_state['State'])
pop_state["Population"] = pop_state["Population"].astype('int')

pop_state = pd.merge(pop_state,geo_state,on=['State'],how='left')
pop_state.drop(columns=["State"],inplace=True)
pop_state.rename(columns={"Area":"State"},inplace=True)
pop_sate = pop_state[["Year","State","Population"]]
pop_sate

pop_state.head()



,Population,Year,State
0,39557045,2018,California
1,39512223,2019,California
2,39237836,2021,California
3,39029342,2022,California


## Pulling in Median Income Data from Census API (ACS)


##### County


In [8]:
## Pulling Median Income By County (CA)

years = [2018,2019,2021,2022] # no 2020 data available due to COVID -19
med_income_var = 'B19013_001E'
base_url = 'https://api.census.gov/data/'
geo = '&for=county:*&in=state:06'  # California = 06
dfs = []

for year in years: 
    url = f'{base_url}{year}/acs/acs1?get={med_income_var}{geo}&key={api_key}'

    data = requests.get(url).json()

    data = pd.DataFrame(data[1:])
   
    data["Year"] = year

    data = data.rename(columns={0: 'Median Income',1: 'State',2:'County'})
    dfs.append(data)

inc_county = pd.concat(dfs, ignore_index=True)
inc_county['State'] = pd.to_numeric(inc_county['State'])
inc_county['County'] = pd.to_numeric(inc_county['County'])
inc_county["Median Income"] = inc_county["Median Income"].astype('int')

inc_county = pd.merge(inc_county,geo_county,on=['State','County'],how='left')
inc_county.drop(columns=["County"], inplace=True)
inc_county.rename(columns={'Area': 'County'},inplace=True)
inc_county['State'] = 'California'
inc_county  = pd.merge(inc_county,state_split, on= "County",how='left')
inc_county = inc_county[['Year', 'State','Region','County','Median Income']]



inc_county.head()



,Year,State,Region,County,Median Income
0,2018,California,NorCal,Lake County,48554
1,2018,California,NorCal,Merced County,57745
2,2018,California,NorCal,Contra Costa County,101618
3,2018,California,NorCal,Stanislaus County,60321
4,2018,California,SoCal,Santa Barbara County,77472


##### State


In [9]:
## Pulling Median Income By State (CA)

years = [2018,2019,2021,2022] # no 2020 data available due to COVID -19
med_income_var = 'B19013_001E'
base_url = 'https://api.census.gov/data/'
geo = '&for=state:06'  # California = 06
dfs = []

for year in years: 
    url = f'{base_url}{year}/acs/acs1?get={med_income_var}{geo}&key={api_key}'

    data = requests.get(url).json()

    data = pd.DataFrame(data[1:])
   
    data["Year"] = year

    data = data.rename(columns={0: 'Median Income',1: 'State',2:'County'})
    dfs.append(data)

inc_state = pd.concat(dfs, ignore_index=True)
inc_state['State'] = pd.to_numeric(inc_state['State'])
inc_state["Median Income"] = inc_state["Median Income"].astype('int')

inc_state = pd.merge(inc_state,geo_state,on=['State'],how='left')
inc_state.drop(columns=["State"],inplace=True)
inc_state.rename(columns={"Area":"State"},inplace=True)
inc_state = inc_state[["Year","State","Median Income"]]
inc_state

,Year,State,Median Income
0,2018,California,75277
1,2019,California,80440
2,2021,California,84907
3,2022,California,91551


## Pulling in Median Home Pricing Data from CSV


In [10]:
#Working with Median Housing Price Data - Cleaning up and getting Mean by year

pd.set_option('display.float_format', '{:.2f}'.format)

csv = "../Josh/Resources/Median(New) - MedianPricesofExistingDetachedHomesHistoricalData.csv"

median_data = pd.read_csv(csv)

median_data = median_data.drop('Unnamed: 54',axis=1)
median_data = median_data.astype('str')

def clean_currency(value):
    try:
        return int(str(value).replace(',', '').replace('$', ''))
    except ValueError:
        # Return nan for non-convertible values
        return np.nan

columns_to_clean = ['CA', 'Alameda', 'Amador', 'Butte', 'Calaveras',
       'Contra-Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin',
       'Mariposa', 'Mendocino', 'Merced', 'Mono', 'Monterey', 'Napa', 'Nevada',
       'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus',
       'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo',
       'Yuba', 'Condo', 'LA Metro', 'Central Coast', 'Central Valley',
       'Far North', 'Inland Empire', 'S.F. Bay Area', 'SoCal']
median_data[columns_to_clean] = median_data[columns_to_clean].applymap(clean_currency)
median_data["Mon-Yr"] = pd.to_datetime(median_data["Mon-Yr"],format='%b-%y')
median_data = median_data[median_data["Mon-Yr"] >= "2018-01-01"]

median_data["Year"] = median_data["Mon-Yr"].dt.year
median_data = median_data[['Year', 'CA', 'Alameda', 'Amador', 'Butte', 'Calaveras',
       'Contra-Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin',
       'Mariposa', 'Mendocino', 'Merced', 'Mono', 'Monterey', 'Napa', 'Nevada',
       'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus',
       'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo',
       'Yuba']]
median_data.reset_index().drop(columns=["index"])
median_grp = median_data.groupby("Year")

median_data = median_grp.mean().reset_index()
median_data


,Year,CA,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,El Dorado,Fresno,...,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba
0,2018,571057.50,926358.33,327476.67,320112.50,321708.33,649169.17,230191.67,491687.42,268630.83,...,667375.00,312658.33,293766.67,219012.50,NaN,232981.67,295350.83,654005.83,439729.17,278813.33
1,2019,591865.83,911500.00,324687.50,361633.33,339101.67,661736.67,261485.83,497453.33,281920.83,...,655243.33,326854.17,310212.50,255875.00,258329.17,243641.67,301125.00,651220.83,452575.83,294937.50
2,2020,650156.67,1001271.67,346366.67,384780.83,366409.17,737000.00,290100.00,529164.17,309070.83,...,695219.17,359583.33,339108.33,276604.17,296495.83,267141.67,317875.00,714145.83,485400.83,333161.67
3,2021,785640.83,1242958.33,421805.83,434448.33,459687.50,902541.67,365845.83,646720.00,367208.33,...,767266.67,425541.67,399816.67,327787.50,327479.17,318210.00,380051.67,819937.50,575560.83,401433.33
4,2022,821204.17,1306546.67,418425.00,446898.33,473070.83,903200.00,375087.50,671083.33,408416.67,...,826887.50,455695.00,431525.00,327394.17,303500.00,357455.83,413500.83,886541.67,623743.33,428329.17
5,2023,811128.89,1221111.11,445243.33,427863.33,480388.89,853632.22,352388.89,653413.33,409376.67,...,831598.89,450738.89,414500.00,321958.89,316343.75,361255.56,407450.00,889444.44,601711.11,435237.78


In [11]:
#melt(take columns and make them rows) Housing Median Price DF into two DFS: One For CA and one by County.  
housing_data = pd.melt(median_data, id_vars=['Year'], var_name='Region', value_name='Median Housing Price')
housing_data["Region"] = housing_data["Region"] + " County"
housing_data.loc[housing_data['Region'] == 'CA County', 'Region'] = 'California'

housing_data.head()


,Year,Region,Median Housing Price
0,2018,California,571057.50
1,2019,California,591865.83
2,2020,California,650156.67
3,2021,California,785640.83
4,2022,California,821204.17


## Creating Final Data Tables for State and County(Merging)


In [12]:
housing_county = housing_data[housing_data['Region']!= 'California'].copy()
housing_county.rename(columns={"Region":"County"},inplace=True)

county_data = pd.merge(inc_county,housing_county,on=["Year","County"])


county_data = pd.merge(pop_county,county_data,on=["Year","State","Region","County"])
county_data.to_csv("../Josh/Clean Data/county_data.csv")  
county_data.head()

,Year,State,Region,County,Population,Median Income,Median Housing Price
0,2018,California,NorCal,Lake County,64382,48554,269408.33
1,2018,California,NorCal,Merced County,274765,57745,268086.67
2,2018,California,NorCal,Stanislaus County,549815,60321,312658.33
3,2018,California,SoCal,Santa Barbara County,446527,77472,662729.17
4,2018,California,NorCal,Sacramento County,1540975,69767,365454.17


In [13]:
housing_state = housing_data[(housing_data['Region']== 'California') & (housing_data['Year'] != 2020) & (housing_data["Year"] != 2023)].copy()
housing_state.rename(columns={"Region":"State"},inplace=True)
state_data = pd.merge(inc_state,housing_state,on=["Year","State"])


state_data = pd.merge(pop_state,state_data,on=["Year","State"])  
state_data= state_data[['Year', 'State','Population', 'Median Income', 'Median Housing Price']]
state_data.to_csv("../Josh/Clean Data/state_data.csv")  

state_data.head()

,Year,State,Population,Median Income,Median Housing Price
0,2018,California,39557045,75277,571057.50
1,2019,California,39512223,80440,591865.83
2,2021,California,39237836,84907,785640.83
3,2022,California,39029342,91551,821204.17
